# Snowpark Traveling Salesman Problem

In [ ]:
# Snowpark Imports
import json
from decimal import Decimal
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

#
import pandas as pd
import networkx as nx
import itertools
from typing import Iterable, Tuple
import matplotlib.pyplot as plt

# Reading Snowflake Connection Details from JSON file
snowflake_connection_cfg = open('../creds.json')
snowflake_connection_cfg = snowflake_connection_cfg.read()
snowflake_connection_cfg = json.loads(snowflake_connection_cfg)

# Creating Snowpark Session
session = Session.builder.configs(snowflake_connection_cfg).create()
print('Current Database:', session.get_current_database())
print('Current Schema:', session.get_current_schema())
print('Current Warehouse:', session.get_current_warehouse())

### Traveling Salesman Problem with NetworkX (Locally)

In [ ]:
# Creating a Graph
graph = nx.Graph()
graph.add_edge('A','B', weight=1)
graph.add_edge('A','H', weight=1)
graph.add_edge('B','C', weight=1)
graph.add_edge('C','D', weight=3)
graph.add_edge('D','E', weight=1)
graph.add_edge('E','F', weight=5) # -> this heavy-weight path should be excluded by the algorithm
graph.add_edge('F','G', weight=1)
graph.add_edge('G','H', weight=1)

# Running the traveling salesman problem algorithm
tsp = nx.approximation.traveling_salesman_problem
# path = tsp(graph, nodes=['C', 'F'], cycle=False) # shortes route from node C to node F
path = tsp(graph, nodes=['A','B','C','D','E','F','G','H'], cycle=False,) # shortest route between all nodes
path_edges = list(zip(path,path[1:]))

# Function to construct list of tuples as return
def ret_tuples(graph,res):
    path_edges = list(zip(res,res[1:]))
    path_edges = [val+(graph.get_edge_data(val[0],val[1])['weight'],) for val in path_edges]
    return path_edges

ret_tuples(graph, path)

In [ ]:
# Visualize Graph
pos = nx.spring_layout(graph)
plt.figure(figsize=(12,6))
nx.draw(
    graph, pos, edge_color='black', width=1, linewidths=1,
    node_size=500, node_color='yellow', alpha=0.9,
    labels={node: node for node in graph.nodes()}
)
# Drawing Shortes Route
nx.draw_networkx_nodes(graph,pos,nodelist=path,node_color='r')
nx.draw_networkx_edges(graph,pos,edgelist=path_edges,edge_color='r',width=10)

# Drawing Edge Labels
edge_labels = dict([((n1, n2), graph.get_edge_data(n1,n2)['weight'])
                    for n1, n2 in graph.edges])
nx.draw_networkx_edge_labels(
    graph, pos,
    edge_labels=edge_labels,
    font_color='blue'
)

# Creating a Snowflake User-Defined-Table-Function (UDTF)

In [ ]:
# Define class for UDTF
class TravelingSalesmanProblem:
    def __init__(self):
        self.graph = nx.Graph()
        
    # In each iteration we are adding edges to our graph
    def process(self, left: str, right: str, weight: int, nodes: list) -> Iterable[Tuple[str, str, int]]:
        self.graph.add_edge(left, right, weight=weight)
        self.nodes = nodes
        return None
        
    # After the graph is built completely we are running the TSP algorithm
    def end_partition(self):
        tsp = nx.approximation.traveling_salesman_problem
        path = tsp(self.graph, nodes=self.nodes, cycle=True)
        res = self.ret_tuples(path)
        return res
    
    # We are returning a list of tuples as a result
    def ret_tuples(self,res):
        path_edges = list(zip(res,res[1:]))
        path_edges = [val+(self.graph.get_edge_data(val[0],val[1])['weight'],) for val in path_edges]
        return path_edges

In [ ]:
# Register UDTF
# We have to include NetworkX as import
# Also, NetworkX requires Numpy
tsp_udtf = session.udtf.register(TravelingSalesmanProblem, ["SRC", "DST", "DISTANCE"], 
                                 name='tsp', 
                                 is_permanent=True, 
                                 stage_location='@MY_FUNCTIONS', 
                                 replace=True, 
                                 packages=['numpy'],
                                 imports=['/opt/conda/envs/pysnowpark/lib/python3.8/site-packages/networkx/'])

In [ ]:
# Generate some Testdata
data = [
    ['GROUP1','A','B', 1],
    ['GROUP1','A','H', 1],
    ['GROUP1','B','C', 1],
    ['GROUP1','C','D', 3],
    ['GROUP1','D','E', 1],
    ['GROUP1','E','F', 5],
    ['GROUP1','F','G', 1],
    ['GROUP1','G','H', 1],
]
data = pd.DataFrame(data, columns=['GROUP', 'LEFT', 'RIGHT', 'DISTANCE_LEFT_RIGHT'])
session.sql('DROP TABLE IF EXISTS TSP_TESTDATA').collect()
session.write_pandas(data, 'TSP_TESTDATA', auto_create_table=True)
df = session.table('TSP_TESTDATA')
df.show()

In [ ]:
# Generate some Testdata
data = [
    ['GROUP1','A','B', 1],
    ['GROUP1','A','H', 1],
    ['GROUP1','B','C', 1],
    ['GROUP1','C','D', 3],
    ['GROUP1','D','E', 1],
    ['GROUP1','E','F', 5],
    ['GROUP1','F','G', 1],
    ['GROUP1','G','H', 1],
]
df = session.create_dataframe(data, schema=['GROUP','LEFT','RIGHT','DISTANCE_LEFT_RIGHT'])
df.show()

In [ ]:
# Calling our UDTF
tsp_result = df.join_table_function(tsp_udtf(df['LEFT'], 
                                             df['RIGHT'],
                                             df['DISTANCE_LEFT_RIGHT'], 
                                             F.array_construct(F.lit('A'),F.lit('D'))).over(partition_by="GROUP", order_by="GROUP"))
tsp_result.select('GROUP','SRC','DST','DISTANCE').show()

### (Real) World Example: Beergarden Route Optimization
Let's try a real world example!<br>
Munich has a lot of places where you can enjoy a cold beer but my time is limited.<br>
Therefore I'd like to get the shortest route that visits all the places and starts/ends from Munich's main station.

In [ ]:
import requests
import urllib.parse
from geojson import Point

# Getting LAT/LON from openstreetmap
def get_point(address):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
    response = requests.get(url).json()
    return Point([float(response[0]["lat"]), float(response[0]["lon"])])

# List of locations
data = [
    ['Munich Main Station','Munich Central Station, 81925, Munich'],
    ['Augustiner-Keller','Arnulfstraße 52, 80335, Munich'],
    ['Chinesischer Turm','Englischer Garten 3 80538, 80538, Munich'],
    ['Seehaus im Englischen Garten','Kleinhesselohe 3, 80802, Munich'],
    ['Hirschgarten','Hirschgarten 1, 80639, Munich'],
    ['Hofbräuhaus','Platzl 9, 80331, Munich'],
    ['Paulaner am Nockherberg','Hochstraße 77, 81541, Munich'],
    ['Paulaner Bräuhaus','Kapuzinerpl. 5, 80337, Munich']
]
data = pd.DataFrame(data, columns=['DESCRIPTION','ADDRESS'])
data['POINT'] = data.apply(lambda row: get_point(row['ADDRESS']), axis='columns')
data

In [ ]:
# Create our dataframe with beergardens
df = session.create_dataframe(data, schema=['DESCRIPTION','ADDRESS','POINT'])
# Transform coordinates to geography data type for distance calculation
df = df.with_column('POINT', F.to_geography('POINT'))
df.toPandas()

In [ ]:
# Get all locations
locations = df.select('DESCRIPTION').collect()

# Loop through all locations and perform cross-joins to retrieve all possible edges
def get_all_edges(locations, df):
    df_locations = None
    for location in locations:
        if df_locations == None:
            # First Location
            df_locations = df.select(F.col('DESCRIPTION').as_("START"), F.col('POINT').as_('POINT1'))
        else:
            # Other Locations
            df_locations = df.cross_join(df.select(F.col('DESCRIPTION').as_("END"), F.col('POINT').as_('POINT2')))
    df_locations = df_locations.select(F.col('DESCRIPTION').as_('START'), 'POINT', 'END', 'POINT2')
    # Remove edges where start and end are the same
    df_locations = df_locations.filter(F.col('START') != F.col('END'))
    return df_locations
res = get_all_edges(locations, df)
# Calculate the distances with builtin function
res = res.with_column('DISTANCE_POINT1_POINT2', F.call_builtin('ST_DISTANCE', F.col('POINT'), F.col('POINT2')))
res = res.with_column('GROUP', F.lit('GROUP1'))
res.toPandas()

In [ ]:
# get all locations in list (first value will be start+end a.k.a circle back home)
locs = [F.lit(val[0]) for val in locations]
print('Start and End will be:', locations[0][0])

# get the shortest route
tsp_result = res.join_table_function(tsp_udtf(res["START"], 
                                             res['END'], 
                                             F.to_decimal(res['DISTANCE_POINT1_POINT2'], 38, 0), 
                                             F.array_construct(*locs)).over(partition_by="GROUP", order_by="GROUP"))
tsp_result = tsp_result.select('GROUP','SRC','DST','DISTANCE')
tsp_result.toPandas()

In [ ]:
# Total Distance
tsp_result.select(F.sum('DISTANCE').as_('TOTAL_DISTANCE')).toPandas()

In [ ]:
# Add addresses and points back to our results
# Left-side (SRC)
finals = tsp_result.join(df, tsp_result.SRC == df.DESCRIPTION)
finals = finals.with_column_renamed(F.col("POINT"), "POINT1")
finals = finals.with_column_renamed(F.col("ADDRESS"), "ADDRESS1")
finals = finals.drop('DESCRIPTION')
# Right-side (DST)
finals = finals.join(df, finals.DST == df.DESCRIPTION)
finals = finals.with_column_renamed(F.col("POINT"), "POINT2")
finals = finals.with_column_renamed(F.col("ADDRESS"), "ADDRESS2")
finals = finals.drop('DESCRIPTION')
finals.toPandas()

In [ ]:
# Visualize data using Google Maps
# Note: Keep in mind that we optimized using ST_DISTANCE which returns the 
# geodesic distance, NOT the real distance using roads.
import gmaps

# Setup API
# https://developers.google.com/maps/documentation/embed/get-api-key -> 200$/Month free usage
api_key = 'AIz....'
gmaps.configure(api_key=api_key)

# Get data to plot (gmaps expects tuples of (lat/lon))
# Transform Points to Lat/Lon
route = finals.select(F.call_builtin('ST_X', F.col('POINT1')).as_('LAT'),
                      F.call_builtin('ST_Y', F.col('POINT1')).as_('LON')).toPandas()
route

# Generate tuples
records = list(route.to_records(index=False))
start = records[0]
end = (start[0], start[1] - 0.001) # easier to visualize with slightly different start/end point
waypoints = records[1:]

# Plot the data
fig = gmaps.figure(map_type='SATELLITE')
beergarden_route = gmaps.directions_layer(
    start, end, waypoints=waypoints,
    travel_mode='WALKING', stroke_opacity=1)
fig.add_layer(beergarden_route)
fig